In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from huggingface_hub import notebook_login
from transformers import GenerationConfig, LlamaForCausalLM, LlamaTokenizer
from peft import PeftModel    

adapters_name  = "experiments/Mistral-7B-v0.1/checkpoint-3000"
model_name = "mistralai/Mistral-7B-v0.1"


print(f"Starting to load the model {model_name} into memory")

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_4bit=True,
    torch_dtype=torch.bfloat16,
    device_map='auto'
)
model = PeftModel.from_pretrained(model, adapters_name) 

tokenizer = LlamaTokenizer.from_pretrained(model_name)


# Define PAD Token = BOS Token
tokenizer.pad_token = tokenizer.bos_token
model.config.pad_token_id = model.config.bos_token_id


Starting to load the model mistralai/Mistral-7B-v0.1 into memory


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
def gen(model, text: str, tmp=.0001, top_p=0.7, max_length=16):
    inputs = tokenizer(text, return_tensors="pt").to('cuda')
    inputs_length = len(inputs["input_ids"][0])
    with torch.inference_mode():
        outputs = model.generate(**inputs, max_new_tokens=max_length,
                                do_sample=True,
                                top_p=top_p,
                                temperature=tmp,
                                repetition_penalty=1.3
                                )

    return tokenizer.decode(outputs[0][inputs_length:], skip_special_tokens=True)


In [10]:
base_text = '''### Instruction:  Below is a clue for a decrypting crossword. Your task is to solve this clue. The number of charachters in the answer should be same as the number in the parenthesis. Just output the answer only.

'''


text= '''### Input:
Celebrity could be made up (6)'''

## solce this clue: "With a degree, I leave this subject" (5)
### aswer: 
##3



with open('mistral_test.txt', 'a') as f:
    while 1:
        print('-'*20)
        # text = input("Enter a prompt: ")
        print('-'*20)

        text = base_text + text

        # text = 'who are you?'
        
        f.write('='*30 + '\n')
        f.write(text + '\n')
        f.write('-'*20 + '\n')


    
        output = gen(model, text)
        output = output.replace('<newline>', '\n')

        f.write(output + '\n')
        f.write('='*30 + '\n')
        f.flush()

        print('-'*20)
        print(output)
        
        print('-'*20)

--------------------
--------------------
--------------------


### Response:
renown,eclipses celebrity (
--------------------
--------------------
--------------------
--------------------


### Response:
renown,eclipsed former leading light
--------------------
--------------------
--------------------
--------------------


### Response:
renown,eclipsed leading celebrity
--------------------
--------------------
--------------------
--------------------


### Response:
renown,eclipsed former leading light
--------------------
--------------------
--------------------


--------------------


### Response:
renown,eclipsed leading American mon
--------------------
--------------------
--------------------
--------------------


### Response:
renown,eclipsed by peer (
--------------------
--------------------
--------------------
--------------------


### Response:
renown,eclipsed former leading light
--------------------
--------------------
--------------------
--------------------


### Response:
renown,eclipsed leading celebrity
--------------------
--------------------
--------------------
--------------------


### Response:
renown,eclipsed by peer having
--------------------
--------------------
--------------------
--------------------


### Response:
renown,eclipsed leading American mon
--------------------
--------------------
--------------------
--------------------


### Response:
renown,e celebrity could be made up
--------------------
--------------------
--------------------
--------------------


### Response:
renown,eclipsed by Yorksh

: 

In [15]:
for i in output_text:

    lines = i.split('\n')
    for i,l in enumerate(lines):
        if l=='### Output:':
            print(lines[i+1])
            
# print(output_text[0])

absent
shocked
shocked
shocked
shocked
shocked
shocked
shocked
shocked


In [8]:
answer_tokens

tensor([[   13,    13,  2277, 29937, 10604, 29901,    13, 29874,   282,  3322,
            13,    13,  2277, 29937, 10567, 29901,    13, 29909,   767, 29915,
         29879,   263,   767,   363,   263, 29915,   393,   313, 29945, 29897,
            13,    13,  2277, 29937, 10604, 29901,    13, 29874,   767,    13,
            13,  2277, 29937, 10567, 29901,    13, 29909,   767, 29915, 29879,
           263,   767,   363,   263, 29915,   393,   313, 29945, 29897,    13,
            13,  2277, 29937, 10604]], device='cuda:0')

In [2]:
import pandas as pd

dataset = pd.read_csv('data/short_list_clues.csv')

dataset['clue']  = dataset['Clue'] 

dataset['clue'].to_csv('clues.csv', index=False, header=True)

In [13]:
from openai import OpenAI, AsyncOpenAI
import asyncio
import os
from utils import get_dataset,get_ans_words_chard
from datasets import load_dataset
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
from tqdm import tqdm   
import emoji
import json


prompt = '''
You are a cryptic crossword expert. The cryptic clue consists of a definition, and a wordplay. The definition usually comes at the beginning or the end of the clue.
The following is a list of some possible wordplay types: 
- anagram:  certain words or letters must be jumbled to form an entirely new term.
    Example clue: Smear pan to cook cheese (8)
    Answer: PARMESAN
- hidden word: the answer will be hidden within one or multiple words within the provided phrase.
    Example clue: Error concealed by city police (4)
    Answer: TYPO
- double definition: a word with two definitions.
    Example clue: Wear out an important part of a car (4)
    Answer: TIRE
- container: the answer is broken down into different parts, with one part embedded within another.
    Example clue: Cursed, being literally last in bed (7)
    Answer: BLASTED
- assemblage: the answer is broken into its component parts and the hint makes references to these in a sequence.
    Example clue: It may get endless representation (5)
    Answer: IMAGE

Solve the following clue, and output only the answer.
Clue: Stipulations for academic periods (5)
Answer:'''

clue_message = {"role": "user", "content": prompt }#clue['prompt']}

model_name = 'gpt-3.5-turbo'
completion = client.chat.completions.create(
model=model_name,
messages=[
    clue_message
],
temperature=0
)

response = completion.choices[0].message.content.lower()

print(response)

terms


: 

In [10]:
response

'anagram'